In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.io as sio
!git clone https://github.com/mcdermottLab/pycochleagram
% cd pycochleagram
!python setup.py install
import pycochleagram.cochleagram as cgram
%cd ..

#Got from Miniplaces 1
!pip install torch
!pip install torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.hub

from os.path import exists


Cloning into 'pycochleagram'...
remote: Enumerating objects: 468, done.
remote: Total 468 (delta 0), reused 0 (delta 0), pack-reused 468
Receiving objects: 100% (468/468), 4.62 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/pycochleagram
running install
running bdist_egg
running egg_info
creating pycochleagram.egg-info
writing pycochleagram.egg-info/PKG-INFO
writing dependency_links to pycochleagram.egg-info/dependency_links.txt
writing requirements to pycochleagram.egg-info/requires.txt
writing top-level names to pycochleagram.egg-info/top_level.txt
writing manifest file 'pycochleagram.egg-info/SOURCES.txt'
adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
adding license file 'AUTHORS' (matched pattern 'AUTHORS*')
writing manifest file 'pycochleagram.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pycochleagram
copying pycochlea

###Get the Zip File:

In [ ]:
!wget -O vis-data.zip https://web.eecs.umich.edu/~ahowens/vis/vis-data.zip?fbclid=IwAR0MTz39Qo4RYz5hCqqfXIUy4uPUBTriqPSj0M1wXR73DqjrzWvCzkT4T5s 

!mkdir video_and_audio

!unzip vis-data.zip -d video_and_audio

!ls
# inside vis-data we have .mat and .pk (visual waveform representation of the audio file)
# ex: 2015-10-06-20-22-06-50_sf.mat, 2015-10-06-20-22-06-50_sf.pk


--2021-05-14 22:18:49--  https://web.eecs.umich.edu/~ahowens/vis/vis-data.zip?fbclid=IwAR0MTz39Qo4RYz5hCqqfXIUy4uPUBTriqPSj0M1wXR73DqjrzWvCzkT4T5s
Resolving web.eecs.umich.edu (web.eecs.umich.edu)... 141.212.113.214
Connecting to web.eecs.umich.edu (web.eecs.umich.edu)|141.212.113.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53835731271 (50G) [application/zip]
Saving to: ‘vis-data.zip’

vis-data.zip        100%[===================>]  50.14G  46.5MB/s    in 14m 36s 

2021-05-14 22:33:26 (58.6 MB/s) - ‘vis-data.zip’ saved [53835731271/53835731271]

Archive:  vis-data.zip
  inflating: video_and_audio/vis-data/2015-02-16-16-49-06_denoised.mp4  
  inflating: video_and_audio/vis-data/2015-02-16-16-49-06_denoised_thumb.mp4  
  inflating: video_and_audio/vis-data/2015-02-16-16-49-06_mic.mp4  
  inflating: video_and_audio/vis-data/2015-02-16-16-56-35_denoised.mp4  
  inflating: video_and_audio/vis-data/2015-02-16-16-56-35_denoised_thumb.mp4  
  inflating: video_

###Will load videos:

In [ ]:
import imageio
import os

!mkdir video_avi

FOLDER_PATH = "video_and_audio/"

for filename in os.listdir(FOLDER_PATH):
  if filename.endswith("_denoised.mp4"): 
    src_dir = FOLDER_PATH + filename
    insert = filename[:len(filename)-4]
    dst_dir = "video_avi/"+insert+".avi"
    reader = imageio.get_reader(src_dir)
    fps = reader.get_meta_data()['fps']
    writer = imageio.get_writer(dst_dir, fps=fps)
    #resizing from (1080, 1920)
    for im in reader:
      writer.append_data(cv2.resize(im[:, :, :], (256,256)))
    writer.close()

#!for i in *.mp4; do ffmpeg -i "$i" -vf scale=256:256 "../video_avi/${i%.*}.avi"; done

vis-data.zip


###Will load Spacetime image sequence

*   video_frames is dictionary with keys as the .avi videos (ex: 2015-02-16-18-02-38_denoised)
* value is a dictionary with first_frame key & frames key





In [ ]:
import os

directory = "video_avi/"

video_frames = {}
for filename in os.listdir(directory):
    if filename.endswith(".avi"): 
         # print(os.path.join(directory, filename))
      store = {}
      frames = []
      cap = cv2.VideoCapture(directory+filename)
      if (cap.isOpened()== False):
        print("Error opening video stream or file")

      while(cap.isOpened()):
        # read frame from the video
        ret, frame = cap.read()
    
        if ret is False:
          break

        if(len(frames) == 0): store["first frame"] = frame #get first frame in rgb
        frames.append(frame)
      
      cap.release()
      frames = np.array(frames, dtype = np.uint8) #/255.
      numFrames = frames.shape[0]
      height = frames.shape[1]
      width = frames.shape[2]
      for i in range(len(frames)):
        if i==0: 
          old = frames[0,:,:,:]
          old = cv2.cvtColor(old, cv2.COLOR_RGB2GRAY)
        else: 
          old = frames[i-1,:,:,1]

        if i==0:
          current = old
        else:
          current = frames[i-1,:,:,2]
        
        if i==(numFrames-1):
          new = current
        else:
          new = frames[i+1,:,:,:]
          new = cv2.cvtColor(new, cv2.COLOR_RGB2GRAY)
          #or is it COLOR_BGR2GRAY?
        ret = np.stack((old,current,new), axis=-1) 
        frames[i,:,:,:] = ret
        
    store["frames"] = frames
    video_frames[filename[:-4]] = store


###Will Load Audio in Cochleagrams:

In [ ]:
import wave
import os

audio_coch = {}
for filename in os.listdir(FOLDER_PATH):
  if filename.endswith("_denoised.wav"): 
    wave_file=wave.open(FOLDER_PATH+filename, "rb")
    frame_rate = wave_file.getframerate()

    # Read file to get buffer                                                                                               
    samples = wave_file.getnframes()
    audio = wave_file.readframes(samples)

        # Convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.uint8)
    #audio_as_np_float32 = audio_as_np_int16.astype(np.float32)

        # (or NOT)Normalise float32 array so that values are between -1.0 and +1.0                                                      
    max_int16 = 2**15
    #audio_normalised = audio_as_np_float32 #/ max_int16
    coch = cgram.human_cochleagram(signal = audio_as_np_int16, nonlinearity = 'power',sr = frame_rate , downsample = 3, ret_mode= 'analytic')
    audio_coch[file_name[:-4]] = coch

#Get Resnet Model's last fully connected layer:

###Download helper

In [ ]:
# Helper function to download things without wget
import requests
def download(url, fn=None):
  if fn is None:
    fn = url.split('/')[-1]
  r = requests.get(url)
  if r.status_code == 200:
      open(fn, 'wb').write(r.content)
      print("{} downloaded: {:.2f} KB".format(fn, len(r.content)/1024.))
  else:
      print("url not found:", url)

In [ ]:
# Initialize the model
resnet = models.resnet50(num_classes=365)
# Download the pretrained weights
#download('http://places2.csail.mit.edu/models_places365/resnet50_places365.pth.tar')
# sd = torch.load('resnet50_places365.pth.tar') ['state_dict'] ## Uncomment this line if you have a GPU or are using Colab
sd = torch.load('resnet50_places365.pth.tar', map_location=torch.device('cpu')) ['state_dict'] ## Uncomment this line if you don't have a GPU

# When a model is trained using parallelism, the weights begin with "module."
# Since we aren't going to be using parallelism, we'll manually change these keys to load the state dict
sd = {k.replace('module.', ''): v for k, v in sd.items()}
resnet.load_state_dict(sd)
resnet.eval()

###Penultimate fc layer:

In [ ]:
print(resnet.conv1.weight.data.size())

###Stuff that I had tried with the Pickle files:

In [ ]:
import _pickle as cPickle
import pickle
import itertools
filename = 'vis-data/2015-10-06-20-22-06-50_sf.pk'
in_s = open(filename, 'rb')
import sys  


def ensure_unicode(v):
    if isinstance(v, str):
        v = v.decode('utf8')
    return unicode(v)

objects = []
with (open("vis-data/2015-10-06-20-22-06-50_sf.pk", "rb")) as openfile:
    while True:
        try:
            objects.append(cPickle.load(openfile, encoding='latin1')) #utf-8-sig
        except EOFError:
            break
# objects is a list with 2 nested lists in it
print(objects)
#print(len(objects)) #1
#print(len(objects[0])) #18
##print(len(objects[0][17])) #45
#print(len(objects[0][14][43])) #42
flat_list  = [item for sublist in objects for item in sublist] #[item for deeplist in objects for sublist in deeplist for item in sublist]
# assuming 

flatter_list  = [item for sublist in flat_list for item in sublist]
#print((flatter_list)) #810

flattest_list  = [item for sublist in flatter_list for item in sublist]
print(len(flattest_list)) #34020
flattest_list = np.array(flattest_list)

nested_list = np.asarray(flatter_list)

#cgram.human_cochleagram( signal = nested_list, sr = strict=False)

NotADirectoryError: ignored

In [ ]:
mat = sio.loadmat('vis-data/2015-10-06-20-22-06-50_sf.mat')
#mat.items()
print(len(mat['sfs']))
print(len(mat['sfs'][0]))
print(len(mat['sfs'][0][0]))
print(mat)


18
45
42
{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Aug 12 22:08:28 2016', '__version__': '1.0', '__globals__': [], 'sfs': array([[[0.02262428, 0.01521877, 0.02153516, ..., 0.03397106,
         0.02769995, 0.03130786],
        [0.01514487, 0.01960596, 0.02411098, ..., 0.03606538,
         0.02561588, 0.03033947],
        [0.01818088, 0.02137686, 0.0249914 , ..., 0.04217081,
         0.02964942, 0.03186953],
        ...,
        [0.01870257, 0.02074585, 0.02713297, ..., 0.08321272,
         0.08515692, 0.0697423 ],
        [0.02265428, 0.02235077, 0.02706844, ..., 0.06754921,
         0.07129256, 0.07154779],
        [0.02321569, 0.02271313, 0.02010071, ..., 0.05943126,
         0.06910671, 0.07054992]],

       [[0.03348286, 0.02349433, 0.02618117, ..., 0.04356711,
         0.04623216, 0.05452603],
        [0.03488976, 0.0230842 , 0.0292407 , ..., 0.04820908,
         0.05346602, 0.05602476],
        [0.0377963 , 0.03311133, 0.03296355, ..., 0.04378672,
     